In [7]:
df = read.csv('../datasets/mediause.csv')
model = lm(formula = 'newspaper ~ age + gender', data = df)
model


Call:
lm(formula = "newspaper ~ age + gender", data = df)

Coefficients:
(Intercept)          age       gender  
   -0.08956      0.06762      0.17666  


In [9]:
gender = c(1,0)
age = c(20,40)
newdata = data.frame(age, gender)
predict(model, newdata)

1        2 
1.439508 2.615248

1        2 
1.439508 2.615248